# 从`LINCS_small_molecules.tsv`中根据`GSE92742_Broad_LINCS_sig_info.txt`里面的BRD标识符匹配化合物SMILES
## 步骤一：准备数据

1. **加载数据**：

   - 从`GSE92742_Broad_LINCS_sig_info.txt`文件加载信号信息数据到`sig_info_df`。
   - 从`LINCS_small_molecules.tsv`文件加载小分子数据到`small_molecules_df`。

## 步骤二：匹配SMILES

1. **合并数据**：
   - 使用`pert_id`作为键，将`sig_info_df`和`small_molecules_df`中的数据合并，保留所有`sig_info_df`中的行，以及`small_molecules_df`中匹配的`canonical_smiles`列。
2. **保存结果**：
   - 将合并后的数据保存到`GSE92742_Broad_LINCS_sig_info_with_smiles.txt`文件中。

## 步骤三：筛选包含BRD的行

1. **加载数据**：
   - 从`GSE92742_Broad_LINCS_sig_info_with_smiles.txt`文件加载已添加SMILES的数据到`filtered_df`。
2. **筛选数据**：
   - 筛选出`pert_id`中包含"BRD"的行。
3. **保存结果**：
   - 将筛选后的数据保存到`GSE92742_Broad_LINCS_sig_info_filtered.txt`文件中。

## 步骤四：筛选有5次以上重复的分子

1. **加载数据**：
   - 从`GSE92742_Broad_LINCS_sig_info_filtered.txt`文件加载筛选后的数据到`replicated_df`。
2. **统计SMILES出现次数**：
   - 统计每个`canonical_smiles`的出现次数。
   - 筛选出出现次数大于5次的SMILES。
3. **筛选数据**：
   - 保留SMILES出现次数大于5次的行。
4. **保存结果**：
   - 将筛选后的数据保存到`GSE92742_Broad_LINCS_sig_info_replicated.txt`文件中。

## 步骤五：验证SMILES的有效性

1. **加载数据**：
   - 从`GSE92742_Broad_LINCS_sig_info_replicated.txt`文件加载数据到`validated_df`。
2. **验证SMILES**：
   - 使用RDKit库验证每个SMILES是否有效。
3. **筛选数据**：
   - 保留SMILES有效的行。
4. **保存结果**：
   - 将验证后的数据保存到`GSE92742_Broad_LINCS_sig_info_final.txt`文件中。

## 步骤六：统计基本信息

1. **加载数据**：
   - 从`GSE92742_Broad_LINCS_sig_info_final.txt`文件加载最终处理后的数据到`sig_info`。
2. **统计信息**：
   - 输出前五行数据。
   - 统计唯一`sig_id`、`pert_id`和`pert_iname`的数量。
   - 统计包含"BRD"的独一无二的`pert_id`数量。


In [2]:

import pandas as pd
from rdkit import Chem

# 文件路径配置
sig_info_path = '/mnt/d/Research/PHD/DLEPS/data/GSE92742/GSE92742_Broad_LINCS_sig_info.txt'
small_molecules_path = '/mnt/d/Research/PHD/DLEPS/data/GSE92742/LINCS_small_molecules.tsv'
output_with_smiles_path = '/mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_with_smiles.txt'
output_filtered_path = '/mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_filtered.txt'
output_replicated_path = '/mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_replicated.txt'
output_final_path = '/mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_final.txt'

def analyze_statistics(file_path):
    """统计文件中的基本信息"""
    df = pd.read_csv(file_path, sep="\t", dtype=str)
    print(f"文件: {file_path}")
    print(f"不重复的实验记录数据(sig_id)数量: {df['sig_id'].nunique()}")
    print(f"不重复的化合物标识号(pert_id)数量: {df['pert_id'].nunique()}")
    print(f"不重复的化合物名称(pert_iname)数量: {df['pert_iname'].nunique()}")
    print(f"不重复的SMILES数量: {df['canonical_smiles'].nunique()}")
    print("-" * 50)

def add_smiles_column():
    """Step 1-3: 匹配 SMILES 并添加到数据"""
    sig_info_df = pd.read_csv(sig_info_path, sep='\t')
    small_molecules_df = pd.read_csv(small_molecules_path, sep='\t')
    merged_df = sig_info_df.merge(small_molecules_df[['pert_id', 'canonical_smiles']], on='pert_id', how='left')
    merged_df.to_csv(output_with_smiles_path, sep='\t', index=False)
    print(f"添加canonical_smiles列后的结果已保存到: {output_with_smiles_path}")
    analyze_statistics(output_with_smiles_path)  # 输出文件统计信息

def filter_brd_data():
    """Step 4-5: 筛选出包含 BRD 的行"""
    merged_df = pd.read_csv(output_with_smiles_path, sep='\t')
    filtered_df = merged_df[merged_df['pert_id'].str.contains('BRD', na=False)]
    filtered_df.to_csv(output_filtered_path, sep='\t', index=False)
    print(f"过滤后的结果（仅保留pert_id包含BRD）已保存到: {output_filtered_path}")
    analyze_statistics(output_filtered_path)  # 输出文件统计信息

def filter_replicates():
    """Step 6: 保留有5次以上重复的分子"""
    filtered_df = pd.read_csv(output_filtered_path, sep='\t')
    
    # 统计每个 SMILES 的出现次数
    smiles_counts = filtered_df['canonical_smiles'].value_counts()
    replicated_smiles = smiles_counts[smiles_counts > 5].index
    replicated_df = filtered_df[filtered_df['canonical_smiles'].isin(replicated_smiles)]
    
    replicated_df.to_csv(output_replicated_path, sep='\t', index=False)
    print(f"筛选后（SMILES 出现次数大于 5）的数据已保存到: {output_replicated_path}")
    analyze_statistics(output_replicated_path)  # 输出文件统计信息

def validate_smiles():
    """Step 7-8: 验证 SMILES 是否有效"""
    replicated_df = pd.read_csv(output_replicated_path, sep='\t')
    
    # 验证 SMILES 是否能被 RDKit 解析
    def is_valid_smiles(smiles):
        try:
            mol = Chem.MolFromSmiles(smiles)
            return mol is not None
        except:
            return False
    
    validated_df = replicated_df[replicated_df['canonical_smiles'].apply(is_valid_smiles)]
    validated_df.to_csv(output_final_path, sep='\t', index=False)
    print(f"最终处理后的数据已保存到: {output_final_path}")
    analyze_statistics(output_final_path)  # 输出文件统计信息



In [3]:
# 调用功能模块
# 注释/取消注释以下功能模块的调用，组合实现不同的功能需求
# 添加 SMILES 列
add_smiles_column()

/home/cpu/anaconda3/envs/dleps/lib/python3.7/site-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.


添加canonical_smiles列后的结果已保存到: /mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_with_smiles.txt
文件: /mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_with_smiles.txt
不重复的实验记录数据(sig_id)数量: 473647
不重复的化合物标识号(pert_id)数量: 51219
不重复的化合物名称(pert_iname)数量: 28927
不重复的SMILES数量: 20314
--------------------------------------------------


In [4]:
# 筛选包含 BRD 的数据
filter_brd_data()

/home/cpu/anaconda3/envs/dleps/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (5,12) have mixed types.Specify dtype option on import or set low_memory=False.
  


过滤后的结果（仅保留pert_id包含BRD）已保存到: /mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_filtered.txt
文件: /mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_filtered.txt
不重复的实验记录数据(sig_id)数量: 204934
不重复的化合物标识号(pert_id)数量: 20403
不重复的化合物名称(pert_iname)数量: 19802
不重复的SMILES数量: 20314
--------------------------------------------------


In [ ]:
# 筛选重复数大于 5 的分子,the molecules must have more than five replicates;
filter_replicates()

筛选后（SMILES 出现次数大于 5）的数据已保存到: /mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_replicated.txt
文件: /mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_replicated.txt
不重复的实验记录数据(sig_id)数量: 169336
不重复的化合物标识号(pert_id)数量: 8978
不重复的化合物名称(pert_iname)数量: 8563
不重复的SMILES数量: 8891
--------------------------------------------------


In [6]:
# 验证 SMILES 是否有效
validate_smiles()

[11:51:23] SMILES Parse Error: syntax error while parsing: -
[11:51:23] SMILES Parse Error: Failed parsing SMILES '-' for input: '-'
[11:51:23] SMILES Parse Error: syntax error while parsing: -
[11:51:23] SMILES Parse Error: Failed parsing SMILES '-' for input: '-'
[11:51:23] SMILES Parse Error: syntax error while parsing: -
[11:51:23] SMILES Parse Error: Failed parsing SMILES '-' for input: '-'
[11:51:23] SMILES Parse Error: syntax error while parsing: -
[11:51:23] SMILES Parse Error: Failed parsing SMILES '-' for input: '-'
[11:51:23] SMILES Parse Error: syntax error while parsing: -
[11:51:23] SMILES Parse Error: Failed parsing SMILES '-' for input: '-'
[11:51:23] SMILES Parse Error: syntax error while parsing: -
[11:51:23] SMILES Parse Error: Failed parsing SMILES '-' for input: '-'
[11:51:23] SMILES Parse Error: syntax error while parsing: -
[11:51:23] SMILES Parse Error: Failed parsing SMILES '-' for input: '-'
[11:51:23] SMILES Parse Error: syntax error while parsing: -
[11:51:2

最终处理后的数据已保存到: /mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_final.txt
文件: /mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_final.txt
不重复的实验记录数据(sig_id)数量: 167430
不重复的化合物标识号(pert_id)数量: 8914
不重复的化合物名称(pert_iname)数量: 8525
不重复的SMILES数量: 8889
--------------------------------------------------
